<a href="https://colab.research.google.com/github/artemisIsNotBeam/enough-lands/blob/main/HowManyLands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**var declaration and imports**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras

import tensorflow as tf
import tensorflow_probability as tfp

train = pd.read_csv("drive/MyDrive/lands.csv")
test = pd.read_csv("drive/MyDrive/testLands.csv")

train60 = pd.read_csv("drive/MyDrive/aiData/60cards.csv")
train100 = pd.read_csv("drive/MyDrive/aiData/comamnder.csv")

In [ ]:
#label is input
#feature is output
def get_features_labels(df):
  #gets the 1st and 2nd column
  features=df.values[:,[0,1]]
  labels = df.values[:,2]

  return features, labels

trainFeatures, trainLabels= get_features_labels(train)
testFeatures, testLabels= get_features_labels(test)

trainFeatures = trainFeatures.astype('int')
trainLabels = trainLabels.astype('int32')


trainBig = pd.read_csv("drive/MyDrive/gmmTrain.csv")

def features_labelsNoCards(df):
  features = df.values[:, np.arange(df.shape[1]) != 1]
  labels = df.values[:,1]

  return features, labels
tf,tl = features_labelsNoCards(trainBig)

tf60,tl60 = features_labelsNoCards(train60)
tf100,tl100 = features_labelsNoCards(train100)

In [ ]:
import sklearn
# let's pull our handy linear fitter from our 'prediction' toolbox: sklearn!
from sklearn.linear_model import LinearRegression

# set up our model
linear60 = LinearRegression()

# train the model
linear60.fit(tf60,tl60)

# test the model
X_new = np.array([[75, 4, 0, 0, 0]])  # create a new data point
y_pred = linear60.predict(X_new)  # predict the label for the new data point

In [ ]:
linear100 = LinearRegression()
linear100.fit(tf100,tl100)

LinearRegression()

**Prediction time!!!**
How to actually use
1. fill out cards in deck
2. fill out toal mana value
3. click run and see the print

In [ ]:
def use100(totmv,fp,rocks,dorks,quickmana):
  input_data = np.array([[totmv, fp, rocks, dorks, 0]])
  value = linear100.predict(input_data)[0]
  rounded = round(value)
  return rounded,value

def use60(totmv,fp,rocks,dorks,quickmana):
  input_data = np.array([[totmv, fp, rocks, dorks, 0]])
  value = linear60.predict(input_data)[0]
  rounded = round(value)
  return rounded,value

print(use100(132,2,0,5,0))

(33, 33.392857142857125)


**flask!**

In [ ]:
!pip install pyngrok
!/ngrok authtoken PASTE_YOUR_AUTHTOKEN_HERE

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=ab1bcf39c2a214b14ef6b2fc224fbed5219b19af08fd39c11d92b3b4cd6e5a8b
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok
/bin/bash: line 1: /ngrok: No such file or directory


In [ ]:
from flask import Flask, render_template,request,url_for,redirect
from pyngrok import ngrok


#SET ngrok auth token
ngrok.set_auth_token("")
publicUrl = ngrok.connect(5000).public_url
app = Flask(__name__, template_folder='drive/MyDrive/website/templates', static_folder="drive/MyDrive/website/static")

@app.route("/")
def home():
  return render_template("home.html")

@app.route("/predict", methods= ['POST', 'GET'])
def predict():
  output = request.form.to_dict()
  if len(output)==0:
    return render_template("goBack.html")

  select_value = request.form.get('select_field')
  check=["mv","freespells","rocks","dorks","fastmana"]
  mv = float(output["mv"])
  fs = float(output["freespells"])
  rocks=float(output["rocks"])
  dorks=float(output["dorks"])
  quickmana = float(output["fastmana"])

  if select_value == "60":
    should,exact = use60(mv,fs,rocks,dorks,quickmana)
  else:
    should,exact = use100(mv,fs,rocks,dorks,quickmana)
  return render_template("predict.html",should = should, exact = exact)

@app.route("/guide")
def guide():
  return render_template("guide.html")
print(f"To acess the link click {publicUrl}")
app.run()

To acess the link click https://850a-35-185-193-185.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Oct/2023 19:43:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Oct/2023 19:43:06] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Oct/2023 19:43:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Oct/2023 19:43:41] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Oct/2023 19:43:42] "GET /static/styles.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [07/Oct/2023 19:43:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Oct/2023 19:43:56] "GET /static/styles.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [07/Oct/2023 19:44:07] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Oct/2023 19:44:07] "GET /static/styles.css HTTP/1.1" 304 -
